**Imports**

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


**Model Class**

In [0]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1, bias=False)     #28*28
        self.bn1   = nn.BatchNorm2d(16) 
        self.d1   = nn.Dropout2d(0.05)

        self.conv2 = nn.Conv2d(16, 24, 3, padding=1, bias=False)    #28*28
        self.bn2   = nn.BatchNorm2d(num_features=24)
        self.d2    = nn.Dropout2d(0.1) 

        self.pool1 = nn.MaxPool2d(2, 2)                 #14*14 

        self.conv11 = nn.Conv2d(24, 16, 1, bias = False)               #14*14
        self.bn11   = nn.BatchNorm2d(num_features=16)

        self.conv3 = nn.Conv2d(16, 20, 3, padding=1, bias=False)    #14*14
        self.bn3   = nn.BatchNorm2d(num_features=20)
        self.d3    = nn.Dropout2d(0.1) 
        
        self.conv4 = nn.Conv2d(20, 30, 3, bias=False)               #12*12
        self.bn4   = nn.BatchNorm2d(num_features=30)
        self.d4    = nn.Dropout2d(0.1) 

        self.pool2 = nn.MaxPool2d(2, 2)                 #6*6

        self.conv5 = nn.Conv2d(30, 20, 3, padding=1)    #6*6
        self.bn5 = nn.BatchNorm2d(num_features=20)
        self.d5    = nn.Dropout2d(0.1) 

        self.conv6 = nn.Conv2d(20, 10, 3, bias=False)               #4*4
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = (self.bn1(F.relu(self.conv1(x))))
        x = self.pool1(self.d2(self.bn2(F.relu(self.conv2(x)))))

        x = self.d3(self.bn3(F.relu(self.conv3(self.bn11(F.relu(self.conv11(x)))))))
        x = self.pool2(self.d4(self.bn4(F.relu(self.conv4(x)))))

        x = self.bn5(F.relu(self.conv5(x)))
        x = (self.conv6(x))

        x = self.avgpool(x)
        x = x.view(-1, 10)
        #print(x.shape)

        return F.log_softmax(x)

**Model Summary**

In [41]:

#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 24, 28, 28]           3,456
       BatchNorm2d-4           [-1, 24, 28, 28]              48
         Dropout2d-5           [-1, 24, 28, 28]               0
         MaxPool2d-6           [-1, 24, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]             384
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 20, 14, 14]           2,880
      BatchNorm2d-10           [-1, 20, 14, 14]              40
        Dropout2d-11           [-1, 20, 14, 14]               0
           Conv2d-12           [-1, 30, 12, 12]           5,400
      BatchNorm2d-13           [-1, 30, 12, 12]              60
        Dropout2d-14           [-1, 30,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


**Load MNIST Data**

In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


**Create Train/Test functions and calculate loss**

In [0]:


from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct1 = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        #bhjh
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred1 = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct1 += pred1.eq(target.view_as(pred1)).sum().item()
        #gdg
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        train_loss, correct1, len(train_loader.dataset),
        100. * correct1 / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

**Model training and validation**

In [45]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(15):
    print('EPOCH: ',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

EPOCH:  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.13038276135921478 batch_id=937: 100%|██████████| 938/938 [00:17<00:00, 52.60it/s]


Train set: Average loss: 0.2200, Accuracy: 56111/60000 (93.52%)




  0%|          | 0/938 [00:00<?, ?it/s]

...


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0159, Accuracy: 9948/10000 (99.48%)

EPOCH:  14


loss=0.0028793364763259888 batch_id=937: 100%|██████████| 938/938 [00:17<00:00, 54.53it/s]


Train set: Average loss: 0.0168, Accuracy: 59686/60000 (99.48%)




Test set: Average loss: 0.0148, Accuracy: 9952/10000 (99.52%)

